# Custom HOG for Ablation Studies


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import Custom_HOGFeatures
import csv
import numpy as np
import os

In [2]:
# Clear folders

for entry in os.scandir("people"):
    if entry.name.endswith(".txt"):
        os.remove(entry.path)
for entry in os.scandir("notpeople"):
    if entry.name.endswith(".txt"):
        os.remove(entry.path)

## Testing Custom_HOGFeatures 
#### NOT the ablation study (scroll down)

In [ ]:
import csv
import cv2
import numpy as np
import os

# ── make sure the custom HoG implementation is on the import path ──────────────
#   (a) copy the class definition you pasted earlier into a file, e.g. custom_hog.py
#   (b) or keep it in the same notebook / script, in which case you can skip the
#       import line below and just instantiate it directly.
from Custom_HOGFeatures import Hog_descriptor          # ← comment-out if not using a module

# ------------------------------------------------------------------------------
# settings
# ------------------------------------------------------------------------------
N_TRAIN_IMAGES   = 5000           # same as numberoftrainingimages
CELL_SIZE        = 8               # ≈ pixels_per_cell=(8,8) in skimage
BIN_SIZE         = 9               # ≈ orientations=9 in skimage
OUT_DIR_PEOPLE   = "people"
OUT_DIR_NOTPEOPLE = "notpeople"

os.makedirs(OUT_DIR_PEOPLE,     exist_ok=True)
os.makedirs(OUT_DIR_NOTPEOPLE,  exist_ok=True)

# ------------------------------------------------------------------------------
# main loop
# ------------------------------------------------------------------------------
with open("train/_annotations.csv", newline="") as f:
    reader = csv.DictReader(f)
    for idx, row in enumerate(reader, start=1):
        if idx > N_TRAIN_IMAGES:                   # honour the original cap
            break

        # --- crop, grayscale, resize ---------------------------------------------------
        img      = cv2.imread(os.path.join("train", row["filename"]))
        crop_img = img[
            int(row["ymin"]):int(row["ymax"]),
            int(row["xmin"]):int(row["xmax"])
        ]
        img_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        img_rs   = cv2.resize(img_gray, (64, 128), interpolation=cv2.INTER_LINEAR)

        # --- custom HoG ---------------------------------------------------------------
        hog = Hog_descriptor(img_rs, cell_size=CELL_SIZE, bin_size=BIN_SIZE)
        fd, hog_vis = hog.extract()       # fd == feature vector, hog_vis == visualisation

        # NOTE: `hog_vis` is already a uint8 image.  The original script saved the
        #       *visualisation* (not fd) as CSV, so we replicate that behaviour.
        #       If you ever need the descriptor instead, convert `fd` to np.ndarray
        #       and save/return it as required.
        hog_vis = np.asarray(hog_vis, dtype=np.float32)

        # --- save visualisation --------------------------------------------------------
        if row["class"].lower() in {"person", "pedestrians"}:
            out_path = os.path.join(OUT_DIR_PEOPLE, f"{idx}_{row['filename']}.txt")
        else:
            out_path = os.path.join(OUT_DIR_NOTPEOPLE, f"{idx}_{row['filename']}.txt")

        np.savetxt(out_path, hog_vis, delimiter=",")

print("Finished extracting and saving custom HoG visualisations!")


Finished extracting and saving custom HoG visualisations!


# Ablation Studies

### Generate HOG list for training

In [3]:
import importlib, ablation_pipeline as hogpipe      # ① import the module
importlib.reload(hogpipe)                           # ② pull in any edits

# ③ overwrite any defaults you care about
hogpipe.CONFIG.update(
    # ── HOG core ────────────────────────────────────────────────────────────
    GRADIENT="sobel",      # sobel | scharr | prewitt | roberts | dog
    UNSIGNED=True,          # 0–180° bins (True) vs. 0–360° (False)
    BIN_SIZE=9,             # number of orientation bins per cell
    CELL_SIZE=8,            # pixels per cell
    BLOCK_NORM="l2hys",     # l2 | l2hys | l1 | none
    PHOG_LEVELS=0,          # add 2 levels of pyramid-HOG (0 = off)

    # ── Pre-processing ─────────────────────────────────────────────────────
    COLOR_SPACE="gray",     # gray | rgb | lab | ycrcb
    GAMMA=1.0,              # 1.0 = off; <1 brightens shadows
    GAUSSIAN_SIGMA=0.0,     # 0.0 = no blur; 0.5–1.0 typical

    # ── Dataset / I/O tweaks ───────────────────────────────────────────────
    MAX_IMAGES=5000,       # cap rows read from the CSV
    SAVE_DESCRIPTOR=True,   # save the 1-D feature vector too -> Need this if using FD like we are
)

# ④ run the extractor
hogpipe.run_for_csv(hogpipe.CONFIG)


Processed 500 images…
Processed 1000 images…
Processed 1500 images…
Processed 2000 images…
Processed 2500 images…
Processed 3000 images…
Processed 3500 images…
Processed 4000 images…
Processed 4500 images…
Processed 5000 images…
Finished extracting!


### Check if it all worked 


In [11]:

def count_files(directory):
    return sum(1 for item in os.listdir(directory) if os.path.isfile(os.path.join(directory, item)))

print(count_files("people"))
print(count_files("notpeople"))

for entry in os.scandir("people"):
    if entry.name.endswith(".txt"):
        print(entry.path)
        print(np.loadtxt(entry.path, delimiter=','))
for entry in os.scandir("notpeople"):
    if entry.name.endswith(".txt"):
        print(entry.path)
        print(np.loadtxt(entry.path, delimiter=','))

218
182
people/91_2008_001401_jpg.rf.5bb5de6318d84a8ebca959693c824f73.jpg.txt
[[ 38.   0.   0. ...   0.   0.   0.]
 [159.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
people/34_2008_003329_jpg.rf.3a95d3718f9de7fc80f69e701acad386.jpg.txt
[[ 13. 129.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
people/27_fourth_053_png_jpg.rf.24eec5b3ef18d64104ca54a302f7f4f1.jpg_fd.txt
[0.12352856 0.05202937 0.06788102 ... 0.         0.         0.        ]
people/148_2008_003510_jpg.rf.ea50b9c86b058b980b4272a1ae41f3de.jpg_fd.txt
[0.04873148 0.131458   0.12896535 ... 0.02327266 0.01053032 0.        ]
people/118_fourth_043_png_jpg.rf.6d217d4ae060a194eb5125ad82762b85.jpg_fd.txt
[0.22937679 0.27786085 0.

#### Generate Tester HOGS

In [4]:
for entry in os.scandir("trainHOG"):
    if entry.name.endswith(".txt"):
        os.remove(entry.path)

In [5]:
# -------------------------------------------------- ① reload module
import importlib, ablation_pipeline as hogpipe
importlib.reload(hogpipe)

import os, csv
from pathlib import Path

# -------------------------------------------------- ② update config (unchanged)
hogpipe.CONFIG.update(
    GRADIENT="sobel",
    UNSIGNED=True,
    BIN_SIZE=9,
    CELL_SIZE=8,
    BLOCK_NORM="l2hys",
    PHOG_LEVELS=0,          # no PHOG
    COLOR_SPACE="gray",
    GAMMA=1.0,
    GAUSSIAN_SIGMA=0.0,
    ANNOT_CSV="test/_annotations.csv",
    TRAIN_DIR="test",
    OUT_PEOPLE="trainHOG",
    OUT_NOTPEOPLE="trainHOG",   # single output folder
    MAX_IMAGES=3000,
    SAVE_DESCRIPTOR=True,       # <-- we really want _fd.txt
)
# -------------------------------------------------- ③ custom extractor
def run_for_csv_test(cfg: dict[str, object]):
    Path(cfg["OUT_PEOPLE"]).mkdir(parents=True, exist_ok=True)

    with open(cfg["ANNOT_CSV"], newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        for idx, row in enumerate(reader, start=1):
            if idx > int(cfg["MAX_IMAGES"]):
                break

            img_path = Path(cfg["TRAIN_DIR"]) / row["filename"]
            img_bgr  = cv2.imread(str(img_path))
            if img_bgr is None:
                continue

            crop = img_bgr[
                int(row["ymin"]): int(row["ymax"]),
                int(row["xmin"]): int(row["xmax"]),
            ]
            if crop.size == 0:
                continue

            img_gray = hogpipe.preprocess(crop, cfg)

            hog = hogpipe.AblationHOG(
                img_gray,
                cell_size=int(cfg["CELL_SIZE"]),
                bin_size=int(cfg["BIN_SIZE"]),
                unsigned=bool(cfg["UNSIGNED"]),
                gradient=str(cfg["GRADIENT"]),
                block_norm=str(cfg["BLOCK_NORM"]),
            )
            hog_fd, hog_vis = hog.extract()

            # ----------- build filename ---------------------------------------
            suffix = "T" if row["class"].lower() in {"person", "pedestrians"} else "F"
            base   = Path(cfg["OUT_PEOPLE"]) / f"{idx}_{row['filename']}{suffix}"

            # ----------- save --------------------------------------------------
            if cfg["SAVE_DESCRIPTOR"]:
                # ❶ descriptor (_fd.txt)
                vec = np.asarray(hog_fd, dtype=np.float32).ravel()
                np.savetxt(f"{base}_fd.txt", vec, delimiter=",")
            else:
                # ❷ visualisation (.txt)
                vis = (hog_vis / 255.0).astype(np.float32)
                np.savetxt(f"{base}.txt", vis, delimiter=",")

# -------------------------------------------------- ④ run it
run_for_csv_test(hogpipe.CONFIG)
